In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w3p2/train_data.csv
/kaggle/input/2022-ai-w3p2/test_data.csv
/kaggle/input/2022-ai-w3p2/sample_submit.csv


In [2]:
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

# 데이터셋 만들기

In [3]:
train = pd.read_csv('../input/2022-ai-w3p2/train_data.csv')
train = train.drop(['ID'],axis=1)
train

,company,model,year,transmission,mileage,fueltype,tax,mpg,engineSize,price
0,2,Focus,2019,1,3764,4,145,34.5,2.3,26299
1,1,I10,2015,1,27056,4,20,60.1,1.0,5495
2,3,A3,2019,3,3375,4,145,34.5,2.0,32400
3,3,A6,2019,0,5743,0,145,47.9,2.0,29990
4,2,Fiesta,2015,1,32782,4,0,65.7,1.0,7250
...,...,...,...,...,...,...,...,...,...,...
17080,0,1 Series,2018,0,4666,4,145,32.5,3.0,23499
17081,3,A4,2020,3,22601,4,145,39.2,2.0,32481
17082,4,Yaris,2018,1,20312,4,145,47.1,1.5,10950
17083,2,Focus,2018,1,18695,4,145,58.9,1.0,13295


In [4]:
X_train = train.iloc[:,:-1]
X_train

,company,model,year,transmission,mileage,fueltype,tax,mpg,engineSize
0,2,Focus,2019,1,3764,4,145,34.5,2.3
1,1,I10,2015,1,27056,4,20,60.1,1.0
2,3,A3,2019,3,3375,4,145,34.5,2.0
3,3,A6,2019,0,5743,0,145,47.9,2.0
4,2,Fiesta,2015,1,32782,4,0,65.7,1.0
...,...,...,...,...,...,...,...,...,...
17080,0,1 Series,2018,0,4666,4,145,32.5,3.0
17081,3,A4,2020,3,22601,4,145,39.2,2.0
17082,4,Yaris,2018,1,20312,4,145,47.1,1.5
17083,2,Focus,2018,1,18695,4,145,58.9,1.0


In [5]:
y_train = train.iloc[:,-1]
y_train

0        26299
1         5495
2        32400
3        29990
4         7250
         ...  
17080    23499
17081    32481
17082    10950
17083    13295
17084     7299
Name: price, Length: 17085, dtype: int64

## 'model' 데이터 실수화가 되어 있지 않으므로 LabelEncoder 함수 이용하여 바꿔주기

In [6]:
model = X_train.loc[:,['model']].to_numpy()

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

model = encoder.fit_transform(model)
model

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([26, 34,  9, ..., 93, 26, 25])

In [7]:
model = pd.DataFrame(model,columns=['model'])
X_train_drop = X_train.drop(['model'],axis=1)

X_train_le = pd.concat([X_train_drop,model],axis=1)
X_train_le

,company,year,transmission,mileage,fueltype,tax,mpg,engineSize,model
0,2,2019,1,3764,4,145,34.5,2.3,26
1,1,2015,1,27056,4,20,60.1,1.0,34
2,3,2019,3,3375,4,145,34.5,2.0,9
3,3,2019,0,5743,0,145,47.9,2.0,12
4,2,2015,1,32782,4,0,65.7,1.0,25
...,...,...,...,...,...,...,...,...,...
17080,0,2018,0,4666,4,145,32.5,3.0,0
17081,3,2020,3,22601,4,145,39.2,2.0,10
17082,4,2018,1,20312,4,145,47.1,1.5,93
17083,2,2018,1,18695,4,145,58.9,1.0,26


## 데이터셋 정규화하기

In [8]:
X_train_le = X_train_le.to_numpy()

In [9]:
from sklearn.preprocessing import MinMaxScaler
scaler1 = MinMaxScaler()

X_train_le = scaler1.fit_transform(X_train_le)
X_train_le

array([[0.5       , 0.33870968, 0.33333333, ..., 0.06773504, 0.36507937,
        0.26804124],
       [0.25      , 0.27419355, 0.33333333, ..., 0.1224359 , 0.15873016,
        0.35051546],
       [0.75      , 0.33870968, 1.        , ..., 0.06773504, 0.31746032,
        0.09278351],
       ...,
       [1.        , 0.32258065, 0.33333333, ..., 0.09465812, 0.23809524,
        0.95876289],
       [0.5       , 0.32258065, 0.33333333, ..., 0.11987179, 0.15873016,
        0.26804124],
       [0.5       , 0.25806452, 0.33333333, ..., 0.15726496, 0.23809524,
        0.25773196]])

In [10]:
y_train = y_train.to_numpy()

In [11]:
scaler2 = MinMaxScaler()

y_train = np.expand_dims(y_train,axis=1)
y_train = scaler2.fit_transform(y_train)
y_train

array([[0.19160813],
       [0.03531047],
       [0.23744412],
       ...,
       [0.07629315],
       [0.09391082],
       [0.04886368]])

## 데이터셋 tensor로 바꿔주기

In [12]:
X_train_le = torch.FloatTensor(X_train_le)
X_train_le

tensor([[0.5000, 0.3387, 0.3333,  ..., 0.0677, 0.3651, 0.2680],
        [0.2500, 0.2742, 0.3333,  ..., 0.1224, 0.1587, 0.3505],
        [0.7500, 0.3387, 1.0000,  ..., 0.0677, 0.3175, 0.0928],
        ...,
        [1.0000, 0.3226, 0.3333,  ..., 0.0947, 0.2381, 0.9588],
        [0.5000, 0.3226, 0.3333,  ..., 0.1199, 0.1587, 0.2680],
        [0.5000, 0.2581, 0.3333,  ..., 0.1573, 0.2381, 0.2577]])

In [13]:
y_train = torch.FloatTensor(y_train)
y_train

tensor([[0.1916],
        [0.0353],
        [0.2374],
        ...,
        [0.0763],
        [0.0939],
        [0.0489]])

In [14]:
#모델초기화
W = torch.zeros([9,1],requires_grad = True)
b = torch.zeros(1,requires_grad = True)

#optiimizer설정
optimizer = optim.Adam([W,b],lr = 1e-5)

#cost
mae = nn.L1Loss()

nb_epochs = 1000000
for epoch in range(nb_epochs+1):
    #H(x) 계산
    hypothesis = X_train_le.matmul(W) + b

    #cost 계산
    cost = mae(hypothesis, y_train)


    #cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch%100000 == 0:
        print('Epoch {:4d}/{} hypothesis: {} Cost:{:.6f}'.format(
            epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
        ))



Epoch    0/1000000 hypothesis: tensor([0., 0., 0.,  ..., 0., 0., 0.]) Cost:0.120392
Epoch 100000/1000000 hypothesis: tensor([0.2127, 0.0412, 0.2006,  ..., 0.1267, 0.0819, 0.0465]) Cost:0.025968
Epoch 200000/1000000 hypothesis: tensor([0.2127, 0.0412, 0.2006,  ..., 0.1267, 0.0819, 0.0465]) Cost:0.025968
Epoch 300000/1000000 hypothesis: tensor([0.2127, 0.0412, 0.2006,  ..., 0.1267, 0.0819, 0.0465]) Cost:0.025968
Epoch 400000/1000000 hypothesis: tensor([0.2127, 0.0412, 0.2006,  ..., 0.1267, 0.0819, 0.0465]) Cost:0.025968
Epoch 500000/1000000 hypothesis: tensor([0.2127, 0.0412, 0.2006,  ..., 0.1267, 0.0819, 0.0465]) Cost:0.025968
Epoch 600000/1000000 hypothesis: tensor([0.2127, 0.0412, 0.2007,  ..., 0.1267, 0.0819, 0.0465]) Cost:0.025968
Epoch 700000/1000000 hypothesis: tensor([0.2127, 0.0412, 0.2006,  ..., 0.1267, 0.0819, 0.0465]) Cost:0.025968
Epoch 800000/1000000 hypothesis: tensor([0.2127, 0.0412, 0.2006,  ..., 0.1267, 0.0819, 0.0465]) Cost:0.025968
Epoch 900000/1000000 hypothesis: ten

In [15]:
print(W)
print(b)

tensor([[ 0.0052],
        [ 0.7906],
        [ 0.0123],
        [-0.0953],
        [ 0.0048],
        [-0.0191],
        [-0.2867],
        [ 0.4611],
        [-0.0011]], requires_grad=True)
tensor([-0.2085], requires_grad=True)


# 모델 테스트하기

In [16]:
X_test = pd.read_csv('../input/2022-ai-w3p2/test_data.csv')
X_test = X_test.drop(['ID'],axis=1)
X_test

,company,model,year,transmission,mileage,fueltype,tax,mpg,engineSize
0,2,Ka+,2017,1,10150,4,145,57.7,1.2
1,2,Kuga,2013,1,50216,0,145,53.3,2.0
2,3,Q5,2016,1,34189,0,200,47.9,2.0
3,1,I10,2020,1,1900,4,145,56.5,1.0
4,0,3 Series,2014,0,79809,0,20,68.9,2.0
...,...,...,...,...,...,...,...,...,...
16826,2,Grand Tourneo Connect,2016,0,29000,0,125,56.6,1.5
16827,4,Yaris,2019,1,2812,4,145,47.9,1.5
16828,2,Kuga,2017,1,46800,0,150,54.3,2.0
16829,3,A7,2017,3,14562,0,145,52.3,3.0


## 'model' 데이터 실수화가 되어 있지 않으므로 LabelEncoder 함수 이용하여 바꿔주기

In [17]:
model2 = X_test.loc[:,['model']].to_numpy()

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

model2 = encoder.fit_transform(model2)
model2

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([44, 46, 60, ..., 46, 13, 59])

In [18]:
model2 = pd.DataFrame(model2,columns=['model'])
X_test_drop = X_test.drop(['model'],axis=1)

X_test_le = pd.concat([X_test_drop,model2],axis=1)
X_test_le

,company,year,transmission,mileage,fueltype,tax,mpg,engineSize,model
0,2,2017,1,10150,4,145,57.7,1.2,44
1,2,2013,1,50216,0,145,53.3,2.0,46
2,3,2016,1,34189,0,200,47.9,2.0,60
3,1,2020,1,1900,4,145,56.5,1.0,34
4,0,2014,0,79809,0,20,68.9,2.0,2
...,...,...,...,...,...,...,...,...,...
16826,2,2016,0,29000,0,125,56.6,1.5,32
16827,4,2019,1,2812,4,145,47.9,1.5,93
16828,2,2017,1,46800,0,150,54.3,2.0,46
16829,3,2017,3,14562,0,145,52.3,3.0,13


## 데이터셋 정규화하기

In [19]:
X_test_le = X_test_le.to_numpy()
X_test_le = scaler1.transform(X_test_le)
X_test_le

array([[0.5       , 0.30645161, 0.33333333, ..., 0.11730769, 0.19047619,
        0.45360825],
       [0.5       , 0.24193548, 0.33333333, ..., 0.10790598, 0.31746032,
        0.4742268 ],
       [0.75      , 0.29032258, 0.33333333, ..., 0.09636752, 0.31746032,
        0.6185567 ],
       ...,
       [0.5       , 0.30645161, 0.33333333, ..., 0.11004274, 0.31746032,
        0.4742268 ],
       [0.75      , 0.30645161, 1.        , ..., 0.10576923, 0.47619048,
        0.13402062],
       [0.75      , 0.25806452, 0.33333333, ..., 0.1017094 , 0.31746032,
        0.60824742]])

## 데이터셋 tensor로 바꿔주기

In [20]:
X_test_le = torch.FloatTensor(X_test_le)
X_test_le

tensor([[0.5000, 0.3065, 0.3333,  ..., 0.1173, 0.1905, 0.4536],
        [0.5000, 0.2419, 0.3333,  ..., 0.1079, 0.3175, 0.4742],
        [0.7500, 0.2903, 0.3333,  ..., 0.0964, 0.3175, 0.6186],
        ...,
        [0.5000, 0.3065, 0.3333,  ..., 0.1100, 0.3175, 0.4742],
        [0.7500, 0.3065, 1.0000,  ..., 0.1058, 0.4762, 0.1340],
        [0.7500, 0.2581, 0.3333,  ..., 0.1017, 0.3175, 0.6082]])

In [21]:
y_test = X_test_le.matmul(W)+b
y_test = y_test.detach().numpy()
y_test

array([[0.08885777],
       [0.0728977 ],
       [0.1223222 ],
       ...,
       [0.12494376],
       [0.22657463],
       [0.07239386]], dtype=float32)

In [22]:
y_test.shape

(16831, 1)

In [23]:
y_test = scaler2.inverse_transform(y_test)
y_test

array([[12622.413],
       [10498.049],
       [17076.697],
       ...,
       [17425.639],
       [30953.217],
       [10430.985]], dtype=float32)

In [24]:
y_test = pd.DataFrame(y_test,columns=['price'])
y_test

,price
0,12622.413086
1,10498.048828
2,17076.697266
3,16289.827148
4,8547.181641
...,...
16826,11515.054688
16827,20532.109375
16828,17425.638672
16829,30953.216797


In [25]:
id = np.arange(16831)
id = pd.DataFrame(id,columns=['id'])
id

,id
0,0
1,1
2,2
3,3
4,4
...,...
16826,16826
16827,16827
16828,16828
16829,16829


In [26]:
result = pd.concat([id,y_test],axis=1)
result

,id,price
0,0,12622.413086
1,1,10498.048828
2,2,17076.697266
3,3,16289.827148
4,4,8547.181641
...,...,...
16826,16826,11515.054688
16827,16827,20532.109375
16828,16828,17425.638672
16829,16829,30953.216797


In [27]:
result.to_csv('submit.csv',index=False)
result

,id,price
0,0,12622.413086
1,1,10498.048828
2,2,17076.697266
3,3,16289.827148
4,4,8547.181641
...,...,...
16826,16826,11515.054688
16827,16827,20532.109375
16828,16828,17425.638672
16829,16829,30953.216797
